In [3]:
from mypipeline import MyPipeline
from diffusers import UNet2DClassConditionModel, DDPMScheduler, AutoencoderKL
import torch
from torchvision.utils import save_image

In [4]:
unet = UNet2DClassConditionModel(
    64,
    cross_attention_dim=512,
    attention_head_dim=1,
    num_class_embeds=2,
    down_block_types=(
        "DownBlock2D",
        "CrossAttnDownBlock2D",
        "CrossAttnDownBlock2D",
        "CrossAttnDownBlock2D",
    ),
    up_block_types=(
        "CrossAttnUpBlock2D",
        "CrossAttnUpBlock2D",
        "CrossAttnUpBlock2D",
        "UpBlock2D",
    ),
    block_out_channels=(192, 384, 576, 960),
    transformer_layers_per_block=(1, 1, 1, 1),
)
unet.load_state_dict(torch.load("results/bean_2/unet.pt"))
unet.to("cuda")

UNet2DClassConditionModel(
  (conv_in): Conv2d(4, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (time_proj): Timesteps()
  (time_embedding): TimestepEmbedding(
    (linear_1): Linear(in_features=192, out_features=768, bias=True)
    (act): SiLU()
    (linear_2): Linear(in_features=768, out_features=768, bias=True)
  )
  (class_embedding): Embedding(3, 512)
  (down_blocks): ModuleList(
    (0): DownBlock2D(
      (resnets): ModuleList(
        (0-1): 2 x ResnetBlock2D(
          (norm1): GroupNorm(32, 192, eps=1e-05, affine=True)
          (conv1): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (time_emb_proj): Linear(in_features=768, out_features=192, bias=True)
          (norm2): GroupNorm(32, 192, eps=1e-05, affine=True)
          (dropout): Dropout(p=0.0, inplace=False)
          (conv2): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (nonlinearity): SiLU()
        )
      )
      (downsamplers): ModuleList(
 

In [5]:
scheduler = DDPMScheduler()

In [6]:
vae = AutoencoderKL.from_pretrained(f"stabilityai/sd-vae-ft-mse").to("cuda")

In [7]:
pipe = MyPipeline(vae=vae, unet=unet, scheduler=scheduler)

In [8]:
class_label = [0, 0,0,0,1,1,1,1]
generator = torch.Generator(device=unet.device).manual_seed(110)
num_inference_steps = 1000
output_type = "tensor"
guidance_scale = 7.0

image_tensor = pipe(
    class_labels=class_label,
    generator=generator,
    num_inference_steps=num_inference_steps,
    output_type=output_type,
).images

100%|██████████| 1000/1000 [01:25<00:00, 11.65it/s]


In [9]:
image_tensor.shape

torch.Size([8, 3, 512, 512])

In [10]:
save_image(image_tensor, "test2.png")

: 